In [1]:
#!/usr/bin/env python
# Use this to visualize and check the correctness of pickled tree files.

import argparse
import pickle
import random

parser = argparse.ArgumentParser()

parser.add_argument(
    "file", type=str, help="The tree pkl file to load and analyze.")


def print_info(tree):
    print("This tree has {} rules".format(len(tree.rules)))
    print("Tree stats: {}".format(tree.compute_result()))
    print("Plottable visualization:\n{}".format(tree.stats_str()))


def check_classification(tree):
    failures = 0
    for i in range(10000):
        if i % 100 == 0:
            print("Testing randomly sampled packets", i)
        if random.random() > 0.5:
            packet = random.choice(tree.rules).sample_packet()
        else:
            packet = (random.randint(0, 2**32 - 1), random.randint(
                0, 2**32 - 1), random.randint(0, 2**16 - 1),
                      random.randint(0, 2**16 - 1), random.randint(
                          0, 2**5 - 1))
        expected_match = None
        for r in tree.rules:
            if r.matches(packet):
                expected_match = r
                break
        actual_match = tree.match(packet)
        expected_match = expected_match and tree.rules.index(expected_match)
        actual_match = actual_match and tree.rules.index(actual_match)
        if expected_match != actual_match:
            print("actual", actual_match, "expected", expected_match)
            failures += 1
    assert failures == 0, failures


def check_invariants(node):
    if node.children:
        if not node.is_partition():
            _check_disjointness(node.children)
        _check_rule_distribution(node)
        for c in node.children:
            check_invariants(c)
    else:
        if len(node.rules) > 16:
            print("WARNING: leaf node found with {} rules".format(
                len(node.rules)))


def _check_rule_distribution(node):
    for r in node.pruned_rules():
        count = 0
        for n in node.children:
            if r in n.rules:
                assert n.is_intersect_multi_dimension(r.ranges)
                count += 1
        if count == 0:
            assert False, ("Rule not found in any children", node.id, r.ranges,
                           [n.ranges for n in node.children])


def _check_disjointness(nodes):
    for ni in nodes:
        for nj in nodes:
            if ni != nj:
                assert not ni.is_intersect_multi_dimension(nj.ranges), \
                    (ni.ranges, nj.ranges)



In [6]:
file_name = 'trees_23_06_23/acl5_1k-7-acc-24-bytes-1592970108.7114053.pkl'
file_name = 'trees_23_06/s_acl_40-3-acc-20-bytes-1592950717.8827252.pkl'
with open(file_name, "rb") as f:
    tree = pickle.load(f)
print_info(tree)
check_invariants(tree.root)
check_classification(tree)

This tree has 40 rules
Tree stats: {'bytes_per_rule': 20.0, 'memory_access': 3, 'num_leaf_node': 20, 'num_nonleaf_node': 2, 'num_node': 22}
Plottable visualization:
widths,1,16,5
dim0,0,0,0
dim1,1,1,0
dim2,0,0,0
dim3,0,0,0
dim4,0,0,0

Testing randomly sampled packets 0
Testing randomly sampled packets 100
Testing randomly sampled packets 200
Testing randomly sampled packets 300
Testing randomly sampled packets 400
Testing randomly sampled packets 500
Testing randomly sampled packets 600
Testing randomly sampled packets 700
Testing randomly sampled packets 800
Testing randomly sampled packets 900
Testing randomly sampled packets 1000
Testing randomly sampled packets 1100
Testing randomly sampled packets 1200
Testing randomly sampled packets 1300
Testing randomly sampled packets 1400
Testing randomly sampled packets 1500
Testing randomly sampled packets 1600
Testing randomly sampled packets 1700
Testing randomly sampled packets 1800
Testing randomly sampled packets 1900
Testing randomly 

In [7]:
dir(tree)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_compute_memory_access',
 '_split',
 'check_contiguous_region',
 'compute_result',
 'create_node',
 'current_node',
 'cut_current_node',
 'cut_current_node_multi_dimension',
 'cut_current_node_split',
 'cut_node',
 'depth',
 'get_current_node',
 'get_depth',
 'get_next_node',
 'get_stats',
 'is_finish',
 'is_leaf',
 'leaf_threshold',
 'match',
 'merge_region',
 'node_count',
 'nodes_to_cut',
 'partition_current_node',
 'partition_cutsplit',
 'partition_efficuts',
 'partition_node',
 'print_layers',
 'print_stats',
 'refinement_equi_dense',
 'refinement_node_merging',
 'refinement_region_compaction',
 'refinement_rule_ove

In [12]:
tree.print_layers()
tree.node_count
tree.compute_result()

Layer 0
ID:0	Action:('cut', 1, 32)	Depth:1	Range:	[0, 4294967296, 0, 4294967296, 0, 65536, 0, 65536, 0, 256]
Children: 1 2 6 7 8 10 11 12 13 16 17 19 20 23 24 25 
Rules:
src_ip:[738520417, 738520418) dst_ip:[759729122, 759729123) src_port:[0, 65536) dst_port:[1550, 1551) proto:[6, 7) 
src_ip:[738520436, 738520437) dst_ip:[1484430176, 1484430177) src_port:[0, 65536) dst_port:[1521, 1522) proto:[6, 7) 
src_ip:[741747831, 741747832) dst_ip:[2418173403, 2418173404) src_port:[0, 65536) dst_port:[1525, 1526) proto:[6, 7) 
src_ip:[741747831, 741747832) dst_ip:[1209321037, 1209321038) src_port:[0, 65536) dst_port:[2121, 2122) proto:[6, 7) 
src_ip:[738520417, 738520418) dst_ip:[1450320456, 1450320457) src_port:[0, 65536) dst_port:[1221, 1222) proto:[6, 7) 
src_ip:[738520417, 738520418) dst_ip:[1450320508, 1450320509) src_port:[0, 65536) dst_port:[30211, 30212) proto:[6, 7) 
src_ip:[741747833, 741747834) dst_ip:[3148974982, 3148974983) src_port:[0, 65536) dst_port:[1526, 1527) proto:[6, 7) 
src_

{'bytes_per_rule': 20.0,
 'memory_access': 3,
 'num_leaf_node': 20,
 'num_nonleaf_node': 2,
 'num_node': 22}